In [1]:
import sys,os

sys.path.append('/home/arash/ProjectVR/')

import importlib
import deepsurv
from statsmodels.stats.outliers_influence import variance_inflation_factor    
import argparse
import uuid
import pickle
import json
import time
import numpy as np
import pandas as pd

import lasagne
import optunity

import logging
from logging import handlers
from sklearn.model_selection import train_test_split

importlib.reload(deepsurv)

from deepsurv import deep_surv, utils

from deepsurv.deepsurv_logger import DeepSurvLogger, TensorboardLogger
from eli5.permutation_importance import get_score_importances
import shap  # package used to calculate Shap values
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

## Load data: 
#### xtrain: 80% of all data, devided into xtrainsub & xvalsub (80 and 20%) 
#### xtest: 20% of all data

In [2]:
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xtrain', 'rb') as f:
    xtrain=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/ytrain', 'rb') as f:
    ytrain=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xtest', 'rb') as f:
    xtest=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/ytest', 'rb') as f:
    ytest=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xtrainsub', 'rb') as f:
    xtrainsub=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/ytrainsub', 'rb') as f:
    ytrainsub=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/xvalsub', 'rb') as f:
    xvalsub=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/yvalsub', 'rb') as f:
    yvalsub=pickle.load(f) 

with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctrain', 'rb') as f:
    NCtrain=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctest', 'rb') as f:
    NCtest=pickle.load(f) 

with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctrain', 'rb') as f:
    NCtrain=pickle.load(f)
    
with open('/home/arash/ProjectVR/cleaneddata/deepwaitdata/nctest', 'rb') as f:
    NCtest=pickle.load(f) 

In [3]:
def load_logger(logdir):
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.DEBUG)
    format = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    
    # Print to Stdout
    ch = logging.StreamHandler(sys.stdout)
    ch.setFormatter(format)
    logger.addHandler(ch)

    # Print to Log file
    fh = logging.FileHandler(os.path.join(logdir, 'log_' + str(uuid.uuid4())))
    fh.setFormatter(format)
    logger.addHandler(fh)

    return logger

def format_to_deepsurv(x, y):
    return {
        'x': x,
        'e': y[:,0].astype(np.int32),
        't': y[:,1].astype(np.float32)
    }

def load_box_constraints(file):
    with open(file, 'rb') as fp:
        return json.loads(fp.read())

def save_call_log(file, call_log):
    with open(file, 'wb') as fp:
        pickle.dump(call_log, fp)

def get_objective_function(num_epochs, logdir, update_fn = lasagne.updates.adam):
    '''
    Returns the function for Optunity to optimize. The function returned by get_objective_function
    takes the parameters: x_train, y_train, x_test, and y_test, and any additional kwargs to 
    use as hyper-parameters.

    The objective function runs a DeepSurv model on the training data and evaluates it against the
    a validation set. The result of the function call is the validation concordance index 
    (which Optunity tries to optimize)
    '''
    def format_to_deepsurv(x, y):
        return {
            'x': x,
            'e': y[:,0].astype(np.int32),
            't': y[:,1].astype(np.float32)
        }

    def get_hyperparams(x,params):
        hyperparams = {
            'batch_norm': True,
            'activation': 'rectify',
            'standardize': False
        }
        params['num_nodes'] = int(np.floor(params['num_nodes']/10))*10    #only 10,20,30 etc are used
        params['dropout'] = int(np.floor(params['dropout']/0.1))*0.1      #only 0, 0.1,0.2 etc are used
        params['hidden_layers_sizes'] = [(params['num_nodes'])] * int(params['num_layers'])
        del params['num_layers']
        del params['num_nodes']

        if 'learning_rate' in params:
            params['learning_rate'] = 10 ** params['learning_rate']
        

        hyperparams.update(params)
        return hyperparams

    def train_deepsurv(x_train,y_train,x_test,y_test,
        **kwargs):
        hyperparams = get_hyperparams(x_train,kwargs)
        hyperparams['n_in'] = xtrain.shape[1]
        x_train=x_train[:,0:hyperparams['n_in']]
        x_test=x_test[:,0:hyperparams['n_in']]
     

        train_data = format_to_deepsurv(x_train, y_train)
        valid_data = format_to_deepsurv(x_test, y_test)

        

        # Set up Tensorboard loggers
        model_id = str(hash(str(hyperparams)))
        run_id = model_id + '_' + str(uuid.uuid4())
        logger = TensorboardLogger('hyperparam_search', 
            os.path.join(logdir,"tensor_logs", model_id, run_id))

        network = deep_surv.DeepSurv(**hyperparams)
        metrics = network.train(train_data, n_epochs = num_epochs, logger=logger, 
            update_fn = update_fn, verbose = False)

        result = network.get_concordance_index(**valid_data)
        main_logger.info('Run id: %s | %s | C-Index: %f | Train Loss %f' % (run_id, str(hyperparams), result, metrics['loss'][-1][1]))
        return result

    return train_deepsurv


## Find the optimal hyper-parameters using training data and save them in opt_params


In [6]:
NUM_EPOCHS = 100
NUM_FOLDS = 8
logdir='/home/arash/ProjectVR/logs'


main_logger = load_logger(logdir)

box_constraints = load_box_constraints('/home/arash/ProjectVR/models/box_constraints.0.json')

In [7]:
opt_fxn = get_objective_function(NUM_EPOCHS, logdir, 
                                 utils.get_optimizer_from_str('adam'))

opt_fxn = optunity.cross_validated(x=xtrain, y=ytrain, num_folds=NUM_FOLDS)(opt_fxn)


opt_params, call_log, _ = optunity.maximize(opt_fxn, num_evals=80,
        solver_name='sobol',
        **box_constraints)

2019-11-14 19:18:05,790 - __main__ - INFO - Run id: 6979678274119849691_4a3ec4e9-3114-47dd-9888-b267e8b96db6 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000862548828125, 'learning_rate': 1.527648498372746e-06, 'dropout': 0.0, 'hidden_layers_sizes': [100], 'n_in': 22} | C-Index: 0.481067 | Train Loss 6.556999
2019-11-14 19:18:05,790 - __main__ - INFO - Run id: 6979678274119849691_4a3ec4e9-3114-47dd-9888-b267e8b96db6 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000862548828125, 'learning_rate': 1.527648498372746e-06, 'dropout': 0.0, 'hidden_layers_sizes': [100], 'n_in': 22} | C-Index: 0.481067 | Train Loss 6.556999
2019-11-14 19:18:20,068 - __main__ - INFO - Run id: 6979678274119849691_37dba725-dfd9-4081-b964-025756f76b87 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000862548828125, 'learning_rate': 1.527648498372746e-06, 'dropout': 0.0, 'hidden_layers_sizes': [100], 'n_i

2019-11-14 19:21:18,623 - __main__ - INFO - Run id: 9154794916250074749_5f856134-569a-48aa-9d98-7d09be2e8615 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.00036754882812500004, 'learning_rate': 0.00014588929899927536, 'dropout': 0.2, 'hidden_layers_sizes': [50, 50, 50], 'n_in': 22} | C-Index: 0.549062 | Train Loss 6.503092
2019-11-14 19:21:18,623 - __main__ - INFO - Run id: 9154794916250074749_5f856134-569a-48aa-9d98-7d09be2e8615 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.00036754882812500004, 'learning_rate': 0.00014588929899927536, 'dropout': 0.2, 'hidden_layers_sizes': [50, 50, 50], 'n_in': 22} | C-Index: 0.549062 | Train Loss 6.503092
2019-11-14 19:21:37,863 - __main__ - INFO - Run id: 9154794916250074749_23e239c6-c893-4634-bca1-d9a8b28d7e20 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.00036754882812500004, 'learning_rate': 0.00014588929899927536, 'dropout': 0.2, 'h

2019-11-14 19:24:15,582 - __main__ - INFO - Run id: 8868046238210498245_99699b25-c679-4786-b09a-2cb9571ac9d1 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000986298828125, 'learning_rate': 0.00045606152748581225, 'dropout': 0.2, 'hidden_layers_sizes': [20, 20, 20], 'n_in': 22} | C-Index: 0.523817 | Train Loss 6.494193
2019-11-14 19:24:15,582 - __main__ - INFO - Run id: 8868046238210498245_99699b25-c679-4786-b09a-2cb9571ac9d1 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000986298828125, 'learning_rate': 0.00045606152748581225, 'dropout': 0.2, 'hidden_layers_sizes': [20, 20, 20], 'n_in': 22} | C-Index: 0.523817 | Train Loss 6.494193
2019-11-14 19:24:33,922 - __main__ - INFO - Run id: 8868046238210498245_edaec377-d9a3-4479-9338-fdbcf9798202 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000986298828125, 'learning_rate': 0.00045606152748581225, 'dropout': 0.2, 'hidden_layers_si

2019-11-14 19:27:19,806 - __main__ - INFO - Run id: -4268749559484175429_ea52f6c7-4414-4a22-8628-1e27b615386f | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000738798828125, 'learning_rate': 4.886787310771984e-07, 'dropout': 0.30000000000000004, 'hidden_layers_sizes': [40], 'n_in': 22} | C-Index: 0.489523 | Train Loss 6.838200
2019-11-14 19:27:32,338 - __main__ - INFO - Run id: -4268749559484175429_9d78979f-309a-4aa5-8e78-df1fb70470d5 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000738798828125, 'learning_rate': 4.886787310771984e-07, 'dropout': 0.30000000000000004, 'hidden_layers_sizes': [40], 'n_in': 22} | C-Index: 0.532470 | Train Loss 6.788578
2019-11-14 19:27:32,338 - __main__ - INFO - Run id: -4268749559484175429_9d78979f-309a-4aa5-8e78-df1fb70470d5 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000738798828125, 'learning_rate': 4.886787310771984e-07, 'dropout': 0.3000

2019-11-14 19:30:31,795 - __main__ - INFO - Run id: -3014412863652490169_79eeba4c-8a78-441f-b1eb-f18daa5d7198 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000243798828125, 'learning_rate': 4.66684565125088e-05, 'dropout': 0.1, 'hidden_layers_sizes': [90, 90], 'n_in': 22} | C-Index: 0.509724 | Train Loss 6.733798
2019-11-14 19:30:31,795 - __main__ - INFO - Run id: -3014412863652490169_79eeba4c-8a78-441f-b1eb-f18daa5d7198 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000243798828125, 'learning_rate': 4.66684565125088e-05, 'dropout': 0.1, 'hidden_layers_sizes': [90, 90], 'n_in': 22} | C-Index: 0.509724 | Train Loss 6.733798
2019-11-14 19:30:51,955 - __main__ - INFO - Run id: 4321520872904549630_eb728cfe-80dd-4774-9179-9f4cd3cd9d00 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000181923828125, 'learning_rate': 2.7009928684364016e-06, 'dropout': 0.1, 'hidden_layers_sizes': [50,

...

2019-11-14 21:57:28,383 - __main__ - INFO - Run id: 4220228197418918080_e7e694a8-e0bf-45eb-881a-d6b2de99a5e9 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000412021484375, 'learning_rate': 0.00022771037185125828, 'dropout': 0.2, 'hidden_layers_sizes': [30, 30], 'n_in': 22} | C-Index: 0.561438 | Train Loss 6.587722
2019-11-14 21:57:28,383 - __main__ - INFO - Run id: 4220228197418918080_e7e694a8-e0bf-45eb-881a-d6b2de99a5e9 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000412021484375, 'learning_rate': 0.00022771037185125828, 'dropout': 0.2, 'hidden_layers_sizes': [30, 30], 'n_in': 22} | C-Index: 0.561438 | Train Loss 6.587722
2019-11-14 21:57:44,599 - __main__ - INFO - Run id: 4220228197418918080_fa832353-9420-4b0d-a049-fd087878bec0 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000412021484375, 'learning_rate': 0.00022771037185125828, 'dropout': 0.2, 'hidden_layers_sizes': [3

2019-11-14 22:00:40,254 - __main__ - INFO - Run id: 6698585572137126704_45bc2632-62c9-4b17-b431-7647b3a6f2fc | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000288271484375, 'learning_rate': 7.453887135290947e-06, 'dropout': 0.0, 'hidden_layers_sizes': [90, 90], 'n_in': 22} | C-Index: 0.496657 | Train Loss 6.734836
2019-11-14 22:00:40,254 - __main__ - INFO - Run id: 6698585572137126704_45bc2632-62c9-4b17-b431-7647b3a6f2fc | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000288271484375, 'learning_rate': 7.453887135290947e-06, 'dropout': 0.0, 'hidden_layers_sizes': [90, 90], 'n_in': 22} | C-Index: 0.496657 | Train Loss 6.734836
2019-11-14 22:00:57,115 - __main__ - INFO - Run id: 6698585572137126704_c50d42b1-da4d-4758-89e3-45bad816172c | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000288271484375, 'learning_rate': 7.453887135290947e-06, 'dropout': 0.0, 'hidden_layers_sizes': [90, 

2019-11-14 22:03:33,040 - __main__ - INFO - Run id: -4973437888806725780_ba143633-86dc-46b6-8542-9c58c043fb08 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.0005357714843750001, 'learning_rate': 7.627510470794888e-07, 'dropout': 0.30000000000000004, 'hidden_layers_sizes': [10, 10, 10], 'n_in': 22} | C-Index: 0.493683 | Train Loss 8.834875
2019-11-14 22:03:33,040 - __main__ - INFO - Run id: -4973437888806725780_ba143633-86dc-46b6-8542-9c58c043fb08 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.0005357714843750001, 'learning_rate': 7.627510470794888e-07, 'dropout': 0.30000000000000004, 'hidden_layers_sizes': [10, 10, 10], 'n_in': 22} | C-Index: 0.493683 | Train Loss 8.834875
2019-11-14 22:03:51,277 - __main__ - INFO - Run id: -4973437888806725780_6802ce09-9d2d-4838-ad52-e054d2085079 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.0005357714843750001, 'learning_rate': 7.6275104707

2019-11-14 22:06:46,234 - __main__ - INFO - Run id: 8110296854181834607_1e48670a-c0cf-4bda-ad83-ea4c32698847 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 4.0771484375000016e-05, 'learning_rate': 7.284215949409998e-05, 'dropout': 0.1, 'hidden_layers_sizes': [60, 60], 'n_in': 22} | C-Index: 0.533514 | Train Loss 6.535184
2019-11-14 22:07:02,835 - __main__ - INFO - Run id: 8110296854181834607_564e2e91-32c6-4df9-8b23-7d1f38f73b11 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 4.0771484375000016e-05, 'learning_rate': 7.284215949409998e-05, 'dropout': 0.1, 'hidden_layers_sizes': [60, 60], 'n_in': 22} | C-Index: 0.534552 | Train Loss 6.482185
2019-11-14 22:07:02,835 - __main__ - INFO - Run id: 8110296854181834607_564e2e91-32c6-4df9-8b23-7d1f38f73b11 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 4.0771484375000016e-05, 'learning_rate': 7.284215949409998e-05, 'dropout': 0.1, 'hidden_layer

2019-11-14 22:10:26,947 - __main__ - INFO - Run id: 6435664314074808278_298bf143-cd53-4ff6-ad9a-b797f8b71388 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000551240234375, 'learning_rate': 0.0008208386784744157, 'dropout': 0.0, 'hidden_layers_sizes': [90, 90], 'n_in': 22} | C-Index: 0.642998 | Train Loss 6.011881
2019-11-14 22:10:26,947 - __main__ - INFO - Run id: 6435664314074808278_298bf143-cd53-4ff6-ad9a-b797f8b71388 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000551240234375, 'learning_rate': 0.0008208386784744157, 'dropout': 0.0, 'hidden_layers_sizes': [90, 90], 'n_in': 22} | C-Index: 0.642998 | Train Loss 6.011881
2019-11-14 22:10:42,621 - __main__ - INFO - Run id: 6435664314074808278_54862fee-a000-4e92-8dca-8875a09776d8 | {'batch_norm': True, 'activation': 'rectify', 'standardize': False, 'lr_decay': 0.000551240234375, 'learning_rate': 0.0008208386784744157, 'dropout': 0.0, 'hidden_layers_sizes': [90, 

In [8]:
opt_params

{'lr_decay': 0.000551240234375,
 'learning_rate': -3.0857421875,
 'num_nodes': 99.02942382812498,
 'num_layers': 2.433291015625,
 'dropout': 0.009347656250000003}

## train a network based on opt_params on training data, test different number of features to be used and check on val set and test set

In [9]:
NUM_EPOCHS=100
hyperparams = {
    'batch_norm': True,
    'activation': 'rectify',
    'standardize': False
}

hyperparams.update(opt_params)


hyperparams['num_nodes'] = int(np.floor(hyperparams['num_nodes']/10))*10
hyperparams['dropout'] = int(np.floor(hyperparams['dropout']/0.1))*0.1
hyperparams['hidden_layers_sizes'] = [(hyperparams['num_nodes'])] * int(hyperparams['num_layers'])
del hyperparams['num_layers']
del hyperparams['num_nodes']

if 'learning_rate' in hyperparams:
    hyperparams['learning_rate'] = 10 ** hyperparams['learning_rate']

    

In [10]:
hyperparams

{'batch_norm': True,
 'activation': 'rectify',
 'standardize': False,
 'lr_decay': 0.000551240234375,
 'learning_rate': 0.0008208386784744157,
 'dropout': 0.0,
 'hidden_layers_sizes': [90, 90]}

In [11]:
# check for different number of features
num=[]
trainacc=[]
valacc=[]
testacc=[]
for i in range(1,22):
    hyperparams['n_in'] = i
    trainsub_data=format_to_deepsurv(xtrainsub, ytrainsub)
    trainsub_data['x']=trainsub_data['x'][:,0:hyperparams['n_in']]
    valsub_data=format_to_deepsurv(xvalsub, yvalsub)
    valsub_data['x']=valsub_data['x'][:,0:hyperparams['n_in']]
    network = deep_surv.DeepSurv(**hyperparams)
    metrics = network.train(trainsub_data,valsub_data, n_epochs = NUM_EPOCHS,
                            update_fn = lasagne.updates.adam, verbose = True)
    
    num.append(i)
    trainacc.append(network.get_concordance_index(trainsub_data['x'],
                                       trainsub_data['t'],trainsub_data['e']))
    
    valacc.append(network.get_concordance_index(valsub_data['x'],
                                       valsub_data['t'],valsub_data['e']))
    
    testacc.append(network.get_concordance_index(xtest[:,0:hyperparams['n_in']],ytest[:,1],ytest[:,0]))

    

In [12]:
C_IndexList = pd.DataFrame(
    {'number of feature': num,
     'train c-index': trainacc,
     'validation c-index': valacc,
     'test c-index':testacc
    })

In [13]:
C_IndexList

,number of feature,train c-index,validation c-index,test c-index
0,1,0.523011,0.534703,0.488173
1,2,0.534797,0.528492,0.506036
2,3,0.534822,0.530896,0.512116
3,4,0.563029,0.560477,0.519781
4,5,0.587693,0.559118,0.515831
5,6,0.605991,0.576053,0.531565
6,7,0.644373,0.611371,0.566908
7,8,0.648770,0.603854,0.566822
8,9,0.681613,0.627067,0.597190
9,10,0.691758,0.603351,0.574756


In [14]:
#choose number of feaures based on best val score
hyperparams['n_in']=int(C_IndexList['number of feature'][C_IndexList['validation c-index'].idxmax()])

In [23]:
#train the final model:
NUM_EPOCHS=100
trainsub_data=format_to_deepsurv(xtrainsub, ytrainsub)
trainsub_data['x']=trainsub_data['x'][:,0:hyperparams['n_in']]
valsub_data=format_to_deepsurv(xvalsub, yvalsub)
valsub_data['x']=valsub_data['x'][:,0:hyperparams['n_in']]
network = deep_surv.DeepSurv(**hyperparams)
metrics = network.train(trainsub_data,valsub_data, n_epochs = NUM_EPOCHS,
                        update_fn = lasagne.updates.adam, verbose = True)

In [24]:
network.get_concordance_index(xtest[:,0:hyperparams['n_in']],ytest[:,1],ytest[:,0])

0.6149099367538784

In [25]:
network.save_model("Deepwait1000epochsTuned", weights_file ="deepwait_weights_1000_epochs")